<a href="https://colab.research.google.com/github/boseull/tensorflow_example/blob/main/tensorflow_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Keras Tuner : TensorFlow 프로그램에 대한 최적의 하이퍼파라미터 세트 선택에 도움을 줌.  
토폴로지 : 컴퓨터 네트워크 요소들을 물리적으로 연결  
하이퍼파라미터 : 훈련 프로세스 및 ML 모델의 토폴로지 제어

*   모델 하이퍼파라미터 : 모델 선택에 영향을 미침
*   알고리즘 하이퍼파라미터 : 학습 알고리즘의 속도와 품질에 영향



##import

In [ ]:
import tensorflow as tf
from tensorflow import keras

import IPython

!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 96 kB 2.7 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import sys


##데이터셋 다운로드


In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

4423680/4422102 [==============================] - 0s 0us/step


##모델 정의하기

모델 아키텍처 + 하이퍼파라미터 검색 공간 정의  
하이퍼 모델 : 하이퍼튜닝을 위해 설정하는 모델  
[방법]

*   모델 빌더 함수 사용
*   Keras Tuner API의 HyperModel 클래스를 하위 클래스화

+HyperXception, HyperResNet



In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  #첫번째 층에서 요소 튜닝, 32-512 사이에서 최적의 값 선택
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compile(optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate), loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

  return model

##튜너를 인스턴스화하고 하이퍼튜닝 수행하기

RandomSearch, Hyperband, BayesianOptimization, Sklearn

In [ ]:
tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=10, factor=3, directory='my_dir', project_name='intro_to_kt')

Hyperband 튜닝 알고리즘 : 적응형 리소스 할당 및 조기 중단 -> 고성능 모델에서 신속하게 수렴  
몇 개의 epoch에 대해 많은 수의 모델 훈련, 최고 성능인 절반만 다음 단계  
1+log_factor(max_epochs) 계산 후 한 브래킷에서 훈련할 모델 수 결정  
하이퍼파라미터 검색 전 훈련 단계가 끝날 때마다 훈련 결과를 지우도록 콜백 정의  

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is {best_hps.get('units')} and the optimal leraning rate for the optimizer is {best_hps.get('learning_rate')}.""")

Trial 30 Complete [00h 01m 22s]
val_accuracy: 0.8849999904632568

Best val_accuracy So Far: 0.8870000243186951
Total elapsed time: 00h 15m 26s
INFO:tensorflow:Oracle triggered exit
The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is 160 and the optimal leraning rate for the optimizer is 0.001.


최적의 하이퍼파라미터로 모델 재훈련

In [ ]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4915 - accuracy: 0.8245 - val_loss: 0.4229 - val_accuracy: 0.8499
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3670 - accuracy: 0.8670 - val_loss: 0.3752 - val_accuracy: 0.8629
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3302 - accuracy: 0.8787 - val_loss: 0.3576 - val_accuracy: 0.8709
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3075 - accuracy: 0.8875 - val_loss: 0.3480 - val_accuracy: 0.8761
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2916 - accuracy: 0.8921 - val_loss: 0.3599 - val_accuracy: 0.8670
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2765 - accuracy: 0.8979 - val_loss: 0.3456 - val_accuracy: 0.8728
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2658 - accuracy: 0.9001 - val_loss: 0.3397 - val_accuracy: